In [24]:
import os
import nltk
import nltk.corpus
from nltk.corpus import stopwords
from nltk.stem import wordnet,WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Innomer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Innomer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [52]:
import pandas as pd
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
import string
import keras
import tensorflow
from keras.layers import Embedding, LSTM, Bidirectional, Dense,Dropout
from keras.models import Sequential
from keras.preprocessing.text import one_hot
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import confusion_matrix

In [26]:
trainds=pd.read_csv('./Datasets/train.csv')

In [27]:
trainds.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [28]:
trainds.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [29]:
trainds.dropna(axis=0,inplace=True)

In [30]:
trainds.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [31]:
trainds

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [32]:
X=trainds.drop('label',axis=1)
y=trainds['label']

In [33]:
messages=X.copy()

In [34]:
messages.drop('id',inplace=True,axis=1)

In [35]:
messages.reset_index(inplace=True)

In [36]:
lemmatizer=WordNetLemmatizer()
corpusTitle=[]
corpusText=[]
for i in range(0,len(messages)):
    s=re.sub('[^a-zA-Z]',' ',messages['title'][i])
    s=s.lower()
    s=s.split()

    s=[lemmatizer.lemmatize(w) for w in s if not w in stopwords.words('english')]
    s=' '.join(s)
    corpusTitle.append(s)
    s=re.sub('[^a-zA-Z0-9]',' ',messages['text'][i])
    s=s.lower()
    s=s.split()

    s=[lemmatizer.lemmatize(w) for w in s if not w in stopwords.words('english')]
    s=' '.join(s)
    corpusText.append(s)

In [37]:
vocab_size=5000

In [38]:
onehot_repr_title=[one_hot(w,vocab_size) for w in corpusTitle]

In [39]:
onehot_repr_text=[one_hot(w,vocab_size) for w in corpusText]

In [40]:
sent_len=20
embedded_title=pad_sequences(onehot_repr_title,padding='post',maxlen=sent_len)

In [41]:
embedded_text=pad_sequences(onehot_repr_text,padding='post',maxlen=sent_len)

In [42]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length=sent_len))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [43]:
X_final=np.array(embedded_title)
y_final=np.array(y)

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [45]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 5s 16ms/step - loss: 0.3206 - accuracy: 0.8436 - val_loss: 0.2036 - val_accuracy: 0.9205
Epoch 2/10
192/192 [==============================] - 3s 15ms/step - loss: 0.1357 - accuracy: 0.9486 - val_loss: 0.2115 - val_accuracy: 0.9188
Epoch 3/10
192/192 [==============================] - 3s 15ms/step - loss: 0.0834 - accuracy: 0.9712 - val_loss: 0.2484 - val_accuracy: 0.9165
Epoch 4/10
192/192 [==============================] - 3s 15ms/step - loss: 0.0511 - accuracy: 0.9805 - val_loss: 0.3096 - val_accuracy: 0.9082
Epoch 5/10
192/192 [==============================] - 3s 15ms/step - loss: 0.0381 - accuracy: 0.9878 - val_loss: 0.4068 - val_accuracy: 0.9059
Epoch 6/10
192/192 [==============================] - 3s 15ms/step - loss: 0.0243 - accuracy: 0.9930 - val_loss: 0.4392 - val_accuracy: 0.9054
Epoch 7/10
192/192 [==============================] - 3s 15ms/step - loss: 0.0177 - accuracy: 0.9947 - val_loss: 0.3686 - val_accuracy: 0.9069

In [51]:
model.save('./Models/LSTM')

INFO:tensorflow:Assets written to: ./Models/LSTM\assets


INFO:tensorflow:Assets written to: ./Models/LSTM\assets


In [47]:
y_pred=model.predict(X_test)
y_pred=np.where(y_pred > 0.6, 1,0)

189/189 [==============================] - 1s 3ms/step


In [48]:
confusion_matrix(y_test,y_pred)

array([[3078,  341],
       [ 225, 2391]], dtype=int64)

In [49]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9062137531068766

In [50]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.90      0.92      3419
           1       0.88      0.91      0.89      2616

    accuracy                           0.91      6035
   macro avg       0.90      0.91      0.90      6035
weighted avg       0.91      0.91      0.91      6035



In [53]:
model=Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length=sent_len))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [54]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 5s 20ms/step - loss: 0.3405 - accuracy: 0.8268 - val_loss: 0.2050 - val_accuracy: 0.9170
Epoch 2/10
192/192 [==============================] - 3s 17ms/step - loss: 0.1518 - accuracy: 0.9411 - val_loss: 0.2122 - val_accuracy: 0.9175
Epoch 3/10
192/192 [==============================] - 3s 17ms/step - loss: 0.1072 - accuracy: 0.9608 - val_loss: 0.2481 - val_accuracy: 0.9163
Epoch 4/10
192/192 [==============================] - 3s 16ms/step - loss: 0.0803 - accuracy: 0.9723 - val_loss: 0.2441 - val_accuracy: 0.9122
Epoch 5/10
192/192 [==============================] - 3s 16ms/step - loss: 0.0586 - accuracy: 0.9802 - val_loss: 0.2906 - val_accuracy: 0.9148
Epoch 6/10
192/192 [==============================] - 3s 16ms/step - loss: 0.0445 - accuracy: 0.9861 - val_loss: 0.3993 - val_accuracy: 0.9112
Epoch 7/10
192/192 [==============================] - 3s 16ms/step - loss: 0.0387 - accuracy: 0.9866 - val_loss: 0.4208 - val_accuracy: 0.9100

In [55]:
model.save('./Models/LSTMDropout')

INFO:tensorflow:Assets written to: ./Models/LSTMDropout\assets


INFO:tensorflow:Assets written to: ./Models/LSTMDropout\assets


In [56]:
y_pred=model.predict(X_test)
y_pred=np.where(y_pred > 0.6, 1,0)

189/189 [==============================] - 1s 3ms/step


In [57]:
confusion_matrix(y_test,y_pred)

array([[3112,  307],
       [ 244, 2372]], dtype=int64)

In [58]:
accuracy_score(y_test,y_pred)

0.9086992543496272

In [59]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.89      0.91      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035



In [60]:
model=Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length=sent_len))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [61]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 11s 41ms/step - loss: 0.3165 - accuracy: 0.8490 - val_loss: 0.1923 - val_accuracy: 0.9215
Epoch 2/10
192/192 [==============================] - 7s 36ms/step - loss: 0.1448 - accuracy: 0.9435 - val_loss: 0.1936 - val_accuracy: 0.9210
Epoch 3/10
192/192 [==============================] - 7s 38ms/step - loss: 0.0979 - accuracy: 0.9651 - val_loss: 0.2199 - val_accuracy: 0.9148
Epoch 4/10
192/192 [==============================] - 7s 39ms/step - loss: 0.0723 - accuracy: 0.9746 - val_loss: 0.2744 - val_accuracy: 0.9160
Epoch 5/10
192/192 [==============================] - 7s 38ms/step - loss: 0.0524 - accuracy: 0.9824 - val_loss: 0.3166 - val_accuracy: 0.9142
Epoch 6/10
192/192 [==============================] - 7s 39ms/step - loss: 0.0400 - accuracy: 0.9878 - val_loss: 0.3405 - val_accuracy: 0.9122
Epoch 7/10
192/192 [==============================] - 7s 37ms/step - loss: 0.0356 - accuracy: 0.9877 - val_loss: 0.3839 - val_accuracy: 0.915

In [62]:
model.save('./Models/BiLSTM')

INFO:tensorflow:Assets written to: ./Models/BiLSTM\assets


INFO:tensorflow:Assets written to: ./Models/BiLSTM\assets


In [63]:
y_pred=model.predict(X_test)
y_pred=np.where(y_pred > 0.6, 1,0)

189/189 [==============================] - 1s 3ms/step


In [64]:
confusion_matrix(y_test,y_pred)

array([[3103,  316],
       [ 213, 2403]], dtype=int64)

In [65]:
accuracy_score(y_test,y_pred)

0.9123446561723281

In [66]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.94      0.91      0.92      3419
           1       0.88      0.92      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

